In [1]:
import pandas as pd
from numpy import array
import numpy as np
import datetime as dt
from scipy import stats
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline

In [2]:
#lOAD HYPSOMETRY DATA AND CREATE DATAFRAME FOR EACH COLUMN
### ENSURE TO EDIT PATHS TO FILES ###
src = 'LOCAL PATH TO GSL_Area_vs_Elevation_dataframe.csv'
df = pd.read_csv(src, dtype='float')
stage = df.iloc[4:, 0:1]
total = df.iloc[4:, 1:2]
SA_FB = df.iloc[4:, 2:3]
NA_BRB = df.iloc[4:, 3:4]
NA_SA = df.iloc[4:, 4:5]
SA = df.iloc[4:, 5:6]
NA = df.iloc[4:, 6:7]
BRB = NA_BRB['NA + BRB']-NA['North Arm Sum']

# DEFINE KNOTS
NA_knots = [4169, 4171, 4173, 4178, 4183, 4188, 4194, 4200, 4201, 4203, 4205, 4207, 4209, 4211, 4214]


#SET UP INTERPOLATION
NA_intrp = LSQUnivariateSpline(stage.to_numpy().T[0], NA.to_numpy().T[0], np.array(NA_knots))

SA_intrp = LSQUnivariateSpline(stage.to_numpy().T[0], SA_FB.to_numpy().T[0], np.array(NA_knots))

BRB_intrp = LSQUnivariateSpline(stage.to_numpy().T[0], BRB.to_numpy(), np.array(NA_knots))

In [3]:
#LOAD csv of elevation data for North and/or South Arms#
# example filenames on github: 'GSL_NA_Elevation_Data_CSV.csv' and'GSL_SA_Elevation_Data_CSV.csv'
NA_elevations = pd.read_csv('LOCAL PATH TO GSL_NA_Elevation_Data_CSV.csv')
NA_elevations['Dates'] = pd.to_datetime(NA_elevations['Date'])
SA_elevations = pd.read_csv('LOCAL PATH TO GSL_SA_Elevation_Data_CSV.csv')
SA_elevations['Dates'] = pd.to_datetime(SA_elevations['Date'])

# Input elevation time series data to output water surface area time series#
NA_area_TS = pd.DataFrame(NA_intrp(NA_elevations['Elevation (ft)'][-650-240:-1])) #you can set the range for which to calculate surface area - in this case I went back to the 1980's to present from the elevation data
NA_dates_TS = pd.to_datetime(NA_elevations['Date'][-650-240:-1])

SA_area_TS = pd.DataFrame(SA_intrp(SA_elevations['Elevation (ft)'][-650-439:-1])) #you can set the range for which to calculate surface area - in this case I went back to the 1980's to present from the elevation data
SA_dates_TS = pd.to_datetime(SA_elevations['Date'][-650-439:-1])

BRB_area_TS = pd.DataFrame(BRB_intrp(SA_elevations['Elevation (ft)'][-650-439:-1])) #you can set the range for which to calculate surface area - in this case I went back to the 1980's to present from the elevation data
BRB_dates_TS = pd.to_datetime(SA_elevations['Date'][-650-439:-1])

#Using the above function to model exposed lakebed area #
NA_intrp_shoreland = pd.DataFrame(2211536846 - 169059598 - NA_area_TS)
SA_intrp_shoreland = pd.DataFrame(2844393367.45774 - SA_area_TS)
BRB_intrp_shoreland = pd.DataFrame(1154526904 - BRB_area_TS)

#voila, now you have dataframes of modelled surface areas from an input time series dataset of lake elevations
# Now go plot and analyze!